# create Json for each survery

0) geo tiff aws ARN
1) polygon bounds
2) survey date      
3) species types
4) species metrics
5) species ID file ARN

In [59]:

import json
import httpx
from shapely.geometry import Polygon

COG_arn = 'https://tiletesting.s3.eu-west-2.amazonaws.com/COG_GDAL_crack_diverModel_Orthomosaic.tif'

COG_arn_species = "https://tiletesting.s3.eu-west-2.amazonaws.com/COG_GDAL_crack_species_only_Orthomosaic_downsample.tif"

json_out_path = "/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/meta_data/ard_bay_2022.json"


titiler_endpoint= "https://os8ci3nx02.execute-api.eu-west-2.amazonaws.com"  # titiler running on AWS Lamda


def whole_geotiff_polygon(url):
    bounded_data = httpx.get(
    f"{titiler_endpoint}/cog/bounds",
    params = {
        "url": url,
        }
    ).json()
    entire_geotiff_poly = Polygon([[bounded_data['bounds'][0],bounded_data['bounds'][1]],
                      [bounded_data['bounds'][2],bounded_data['bounds'][1]],
                      [bounded_data['bounds'][2],bounded_data['bounds'][3]],
                      [bounded_data['bounds'][0],bounded_data['bounds'][3]]])
    return entire_geotiff_poly
 
def get_centre_geotiff(url):
    bounded_data = httpx.get(
    f"{titiler_endpoint}/cog/bounds",
    params = {
        "url": url,
        }
    ).json()
    center_lon = (bounded_data['bounds'][2]+bounded_data['bounds'][0])/2
    center_lat = (bounded_data['bounds'][1]+bounded_data['bounds'][3])/2
    centre_coords = [center_lon,center_lat]
    return centre_coords



# Data to be written
dictionary = {
    "survey_dash_id": "ard_bay_2022",
    "survey_long_name": "Ardmucknish Bay 2022",
    "COG_ARN" : COG_arn,
    "COG_ARN_species" : COG_arn_species,
    "COG_species_filename" : "/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/data/crack_species_only_Orthomosaic_downsample.tif" , 
    "survey_date": '01/01/2022',
    "species_name": "Alcyonium digitatum",
    "number_of_individuals" : 'N/A',
    'species_m2':8,
    "centre_coords" :list(reversed(get_centre_geotiff(COG_arn))),
    "nat_cap_potential_description" : 'Medium',
    "nat_cap_number" : 50,
    "nat_cap_value" : ' 0.5 million dollars'
    
}
 
# Serializing json
json_object = json.dumps(dictionary, indent=4)
 
# Writing to sample.json
with open(json_out_path, "w") as outfile:
    outfile.write(json_object)



In [61]:
import json
import httpx
from shapely.geometry import Polygon

COG_arn = 'https://tiletesting.s3.eu-west-2.amazonaws.com/COG_GDAL_CrereanSerpOrthoCropped.tif'

COG_arn_species = "https://tiletesting.s3.eu-west-2.amazonaws.com/COG_GDAL_CreranSerpClassified_downsample.tif"

json_out_path = "/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/meta_data/creran_2023.json"


titiler_endpoint= "https://os8ci3nx02.execute-api.eu-west-2.amazonaws.com"  # titiler running on AWS Lamda


def whole_geotiff_polygon(url):
    bounded_data = httpx.get(
    f"{titiler_endpoint}/cog/bounds",
    params = {
        "url": url,
        }
    ).json()
    entire_geotiff_poly = Polygon([[bounded_data['bounds'][0],bounded_data['bounds'][1]],
                      [bounded_data['bounds'][2],bounded_data['bounds'][1]],
                      [bounded_data['bounds'][2],bounded_data['bounds'][3]],
                      [bounded_data['bounds'][0],bounded_data['bounds'][3]]])
    return entire_geotiff_poly
 
def get_centre_geotiff(url):
    bounded_data = httpx.get(
    f"{titiler_endpoint}/cog/bounds",
    params = {
        "url": url,
        }
    ).json()
    center_lon = (bounded_data['bounds'][2]+bounded_data['bounds'][0])/2
    center_lat = (bounded_data['bounds'][1]+bounded_data['bounds'][3])/2
    centre_coords = [center_lon,center_lat]
    return centre_coords



# Data to be written
dictionary = {
    "survey_dash_id": "creran_2023",
    "survey_long_name": "Creran 2023",
    "COG_ARN" : COG_arn,
    "COG_ARN_species" : COG_arn_species,
    "COG_species_filename" : "/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/ortho_geotiff/CreranSerpClassified_downsample.tif" , 
    "survey_date": '01/02/2023',
    "species_name": "Serpula vermicularis",
    "number_of_individuals" : 31,
    'species_m2':4,
    "centre_coords" :list(reversed(get_centre_geotiff(COG_arn))),
    "nat_cap_potential_description" : 'High',
    "nat_cap_number" : 100,
    "nat_cap_value" : ' 1 million dollars'

    
}
 
# Serializing json
json_object = json.dumps(dictionary, indent=4)
 
# Writing to sample.json
with open(json_out_path, "w") as outfile:
    outfile.write(json_object)


In [7]:
import os

os.system(' gdalwarp -of COG  -co COMPRESS=DEFLATE /Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/ortho_geotiff/CreranSerpClassified_downsample.tif /Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/ortho_COG/COG_GDAL_CreranSerpClassified_downsample.tif')

Using band 4 of source image as alpha.
Creating output file that is 5000P x 1420L.
0...10...20...30...40...50...60...70...80...90...100 - done.


0

In [55]:
import json
import pandas as pd
# survey json file list
survey_json_paths = ['/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/meta_data/ard_bay_2022.json', 
                     '/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/meta_data/creran_2023.json',
                    
]

# load all jsons could maybe load on demand in future
survey_json_list = []
for  survey in survey_json_paths:
    with open(survey, 'r') as openfile:
        survey_json = json.load(openfile)
    survey_json_list.append(survey_json)
survey_df = pd.DataFrame.from_dict(survey_json_list)



if survey_df.loc[survey_df['species_name'] == 'Alcyonium digittum']['survey_dash_id'].values == 'ard_bay_2022' :
    print('yes')

# ['survey_dash_id'].values[0] == 'ard_bay_2022'

# ['survey_dash_id'].values[0] == 'ard_bay_2022'

coords = list(reversed(survey_df.loc[survey_df['survey_dash_id'] == 'ard_bay_2022']['centre_coords'].values[0]))
print(coords)

[56.45732135439754, -5.4354643821716255]


/var/folders/_h/fzs7y34n0fn5jt_snsc_tvym0000gn/T/ipykernel_61106/2180653352.py:19: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if survey_df.loc[survey_df['species_name'] == 'Alcyonium digittum']['survey_dash_id'].values == 'ard_bay_2022' :
